## Imports

In [1]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite.sqlite_copy')

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

In [3]:
request = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(test.first_commit_ts) - 
                    JulianDay(datetime(deadlines.deadlines, 'unixepoch'))
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
             ELSE 'after' END AS time
       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                           ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(request, connect)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [4]:
request = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(control.first_commit_ts) -
                    JulianDay(datetime(deadlines.deadlines, 'unixepoch'))
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
             ELSE 'after'
             END AS time
             FROM control
             LEFT JOIN deadlines ON control.labname=deadlines.labs
             WHERE labname <> 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                           ELSE 'after'
                           END AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(request, connect)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


## we still are not using the lab ’project1’

## close the connection

In [5]:
connect.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

yes